In [ ]:
import json
import requests
from src import utils
import time
import os

In [ ]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [ ]:
api = utils.read_json_file(api_path)

In [ ]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'result-' + input_file.split('/')[-1]
output_file = 'zero-shot-cot-result-ko-gsm8k-test.jsonl'
print(f'output_file : {output_file}')

In [ ]:
output_file = output_path + '/' + 'test.jsonl'
print(f'output_file : {output_file}')

In [ ]:
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
data = utils.read_jsonlines(input_file)

In [ ]:
len(data)

In [ ]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [ ]:
model = model_api(api['url'], api['headers'])

In [ ]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [ ]:
# 차근차근 -> 단답 비율 높음
### 차근차근 생각해보자 0
### 차근 차근 풀어보면 0
### 단계적으로 생각해보면 1(reasoning/caculation 둘다 오답)
### 단계적으로 생각하면
### 단계별로 생각해보면 

# 단계벌 -> 단계별 접근
# 단계별로 질문에 대답해보면 0
# 단계별로 질문에 접근해보면 0

# 각 단계가 숫자로 구분되어짐 (음식의 경우, 조리방법)
# 단계별로 질문을 분석하면 0
# 질문을 단계적으로 풀어보면, 0



 

def get_gsm8k_zs_cot(que):
    prompt = f"질문: {que} 정답: 질문을 단계적으로 풀어보면, "
    return prompt

In [ ]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [ ]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [ ]:
cnt = 0
split_cnt = 0
for ins in data:
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs_cot(kor_que)
    ans_s = ins['answer_s']
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print(f'ans_s : ')
    print(ans_s)
    print('----')
    
    ins['kor_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 20:
        break
    else:
        cnt += 1